### Data Pre-Processing

This notebook will deal with the data preprocessing for CounselChat followed by little visualization related to data statistics:

1. Remove any non-unicode characters from data e.g., non-breakable spaces, smileys, etc.
2. Remove all the rows having empty questions/answers
3. Get the distribution of the answers length in the dataset
4. Map the answers with the votes to see if long answers are getting more number of votes
5. Save the filtered dataset (question/answer pairs) to be used for further inference and/or evaluation

In [1]:
from datasets import load_dataset
import pandas as pd
import pickle
import matplotlib.pyplot as plt

/data/mn27889/miniconda3/envs/mental-health/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Get the CounselChat Dataset

In [2]:
dataset_name = "nbertagnolli/counsel-chat"
dataset = load_dataset(dataset_name, split="all")
dataset = dataset.shuffle(seed=42)

Repo card metadata block was not found. Setting CardData to empty.


In [5]:
dataset_df = dataset.to_pandas()
dataset_df.head()

,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views
0,569,How do you get over someone cheating on you wh...,My boyfriend of five years told me he cheated ...,https://counselchat.com/questions/how-do-you-g...,intimacy,"Lynda MartensMarriage & Family Therapist, MSc,...",https://counselchat.com/therapists/ms-lynda-ma...,"Hi Michigan,This is a common issue. How do you...",2,130
1,787,Is it acceptable for me to find another home f...,My boyfriend moved in with me a few months ago...,https://counselchat.com/questions/is-it-accept...,relationships,"Sherry Katz, LCSWCouples and Family Therapist,...",https://counselchat.com/therapists/sherry-katz...,Every living creature needs time and support t...,2,66
2,229,Am I somehow stressing myself out?,"I think I'm annoying and bothersome, but my gi...",https://counselchat.com/questions/am-i-somehow...,anxiety,"Lynda MartensMarriage & Family Therapist, MSc,...",https://counselchat.com/therapists/ms-lynda-ma...,"""Where you put your attention is where you wil...",2,271
3,933,How would I know if I have the right therapist?,How do you know you have the right therapist f...,https://counselchat.com/questions/how-would-i-...,counseling-fundamentals,"Jane M Floyd, PsyDOur life is shaped by our mi...",https://counselchat.com/therapists/jane-m-floy...,A therapist should be willing to have an initi...,0,29
4,929,Why can't I stop these thoughts?,I keep having these random thoughts that I don...,https://counselchat.com/questions/why-can-t-i-...,diagnosis,"Sherry Katz, LCSWCouples and Family Therapist,...",https://counselchat.com/therapists/sherry-katz...,Those critical thoughts most likely came into...,0,740
